In [47]:
import numpy as np

from anony_process_pipeline import *
from src.modules.utils_image import pillow_image_to_bytes, numpy_image_to_pillow
from database_controller import DatabaseController


In [48]:
from werkzeug.datastructures import FileStorage
import os

def import_subject_images(ids):
    image_folder = "../../data/yalefaces"
    sujet = f"subject0{ids}_"
    image_files = []
    for filename in os.listdir(image_folder):
        if filename.startswith(sujet):
            chemin_fichier = f"{image_folder}/{filename}"
            file = open(chemin_fichier, "rb")
            file_storage = FileStorage(stream=file, filename=filename)
            image_files.append(file_storage)
    return image_files

# Import images
img_list = import_subject_images(3)

# Print result
img_list

[<FileStorage: 'subject03_centerlight.png' (None)>,
 <FileStorage: 'subject03_glasses.png' (None)>,
 <FileStorage: 'subject03_happy.png' (None)>,
 <FileStorage: 'subject03_leftlight.png' (None)>,
 <FileStorage: 'subject03_noglasses.png' (None)>,
 <FileStorage: 'subject03_normal.png' (None)>,
 <FileStorage: 'subject03_rightlight.png' (None)>,
 <FileStorage: 'subject03_sad.png' (None)>,
 <FileStorage: 'subject03_sleepy.png' (None)>,
 <FileStorage: 'subject03_surprised.png' (None)>,
 <FileStorage: 'subject03_wink.png' (None)>]

In [49]:
# ---------------------------------------
# Étape 1 : Preprocessing (Simplifiée)
# ---------------------------------------
res1 = run_preprocessing(filestorage_list=img_list)

# IMG IMPORT ENDING: Realise images in storage ###
for img_in_storage in img_list: ##################
    img_in_storage.close() #######################
##################################################

# Print result
res1

2025-04-05 20:36:47,494 - INFO - Exécution du pré-traitement standard...
2025-04-05 20:36:47,496 - INFO - Traitement de 11 fichiers FileStorage.
Preprocessing (FileStorage List): 100%|██████████| 11/11 [00:00<00:00, 332.50it/s]
2025-04-05 20:36:47,534 - INFO - Pré-traitement terminé. 1 sujets traités.


{'1': [{'resized_image': <PIL.Image.Image image mode=RGB size=100x100>,
   'grayscale_image': <PIL.Image.Image image mode=L size=100x100>,
   'normalized_image': array([[0.81960784, 0.81960784, 0.81960784, ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          ...,
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           0.98823529],
          [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
           1.        ],
          [0.73333333, 0.73333333, 0.73333333, ..., 0.73333333, 0.73333333,
           0.73333333]]),
   'flattened_image': array([0.81960784, 0.81960784, 0.81960784, ..., 0.73333333, 0.73333333,
          0.73333333]),
   'imageId': 'upload_img_0'},
  {'resized_image': <PIL.Image.Image image mode=RGB size=100x100>,
   'gray

In [50]:
# ---------------------------------------
# Étape 2 : K-Same-Pixel (Nouvelle Fonction Modulaire)
# ---------------------------------------
k_same_k_value = 4
res2 = run_k_same_anonymization(res1, k_same_k_value)

# Print result
res2

2025-04-05 20:36:50,947 - INFO - Application de K-Same-Pixel avec k=4...
K-Same Pixel: 100%|██████████| 1/1 [00:00<00:00, 323.46it/s]
2025-04-05 20:36:50,959 - INFO - Traitement K-Same-Pixel terminé.


{'1': [array([[191, 195, 192, ..., 252, 252, 254],
         [235, 234, 234, ..., 254, 250, 250],
         [233, 231, 236, ..., 254, 249, 245],
         ...,
         [218, 213, 208, ..., 243, 243, 241],
         [212, 208, 205, ..., 244, 243, 243],
         [157, 155, 154, ..., 181, 179, 177]], dtype=uint8),
  array([[209, 209, 209, ..., 248, 247, 240],
         [255, 255, 255, ..., 245, 249, 245],
         [255, 255, 255, ..., 245, 249, 249],
         ...,
         [255, 255, 255, ..., 197, 197, 197],
         [255, 255, 255, ..., 197, 196, 196],
         [187, 187, 187, ..., 150, 150, 150]], dtype=uint8),
  array([[209, 209, 209, ..., 248, 247, 240],
         [255, 255, 255, ..., 245, 249, 245],
         [255, 255, 255, ..., 245, 249, 249],
         ...,
         [255, 255, 255, ..., 197, 197, 196],
         [255, 255, 255, ..., 197, 196, 196],
         [187, 187, 187, ..., 150, 150, 150]], dtype=uint8),
  array([[209, 209, 209, ..., 248, 247, 240],
         [255, 255, 255, ..., 245,

In [79]:
# from k same pixel
n_components_ratio = 0.9
user = list(res2.keys())[0]
original_image_ids = res2[user]
original_image_ids = [img.flatten() for img in original_image_ids]
original_image_ids = np.array(original_image_ids)
original_image_ids.shape

(11, 10000)

In [80]:
# from preprocess
n_components_ratio = 0.9
subject_id = list(res1.keys())[0]
subject_preprocessed_list = res1[subject_id]

original_image_ids = [img_dict.get('flattened_image') for img_dict in subject_preprocessed_list]
original_image_ids = np.array(original_image_ids)
original_image_ids.shape


(11, 10000)

In [84]:
# ---------------------------------------
# Étape 3 : PEEP Eigenface
# ---------------------------------------

n_samples, n_features = original_image_ids.shape
n_components = min(max(1, int(n_components_ratio * n_samples)), n_features, n_samples) # Assurer >= 1
res3 = run_eigenface(original_image_ids, 4)

# Print result
pca, mean_face, projection = res3 # Eingenface a disparue...
print("n_components:", n_components)
projection

2025-04-05 20:59:37,283 - DEBUG - Calcul Eigenfaces avec n_components=4


n_components: 9


array([[ -5.10157982,   5.933605  ,  13.03178282,   3.81468684],
       [ -0.04764854,   3.48412747,  12.45421854,   1.29498873],
       [  8.79449306,  -0.93757355,  -2.20666091,  -3.51201188],
       [-26.73328319,  27.6321078 ,  -6.93863203,  -0.69504606],
       [  9.46337868,  -0.88573873,  -2.45088113,  -2.30966492],
       [  9.46337868,  -0.88573873,  -2.45088113,  -2.30966492],
       [-28.41025716, -29.42548463,  -1.21325936,  -1.00278734],
       [  7.74196461,  -0.21524605,  -0.24775197,  -1.26366732],
       [  8.29072216,  -0.31235984,  -0.90659751,  -4.02532372],
       [  8.71511556,  -3.65490237,  -7.50013985,  13.3561612 ],
       [  7.82371597,  -0.73279636,  -1.57119747,  -3.34767061]])

In [86]:
# Étape 4 : Ajout de bruit différentiel
epsilon = 0.01
res4 = run_add_noise(projection, epsilon, sensitivity=1.0)

# Étape 5 : Reconstruction
res5 = run_reconstruction(pca, res4)

# Print result
import base64
from io import BytesIO
from PIL import Image
imgs = [Image.open(BytesIO(base64.b64decode(img))) for img in res5]
print(len(imgs))
imgs[0].show()
res5

2025-04-05 21:13:10,120 - DEBUG - Ajout bruit Laplacien (epsilon=0.01, sensitivity=1.0)
2025-04-05 21:13:10,124 - DEBUG - Reconstruction des images depuis projection bruitée...


11


['...']

In [17]:
db = DatabaseController(r"..\..\data\database.db")

user_id = db.add_user(np.array(res5))
print(f"user_id: {user_id}")

user = db.get_user(user_id)

base64_str = user[0]
image_bytes = base64.b64decode(base64_str)
image = Image.open(io.BytesIO(image_bytes))
image.show()

Create db connection at: D:\myDocs\Apprentissage\Isep\Year_3\Brno-Privacy_Preserving_Face_Recognition_Project\data\database.db
user_id: 4
